In [2]:
import sqlite3
import pandas as pd
import xml.etree.ElementTree as ET
import http.client
import time

conn = http.client.HTTPSConnection("api.sportradar.us")

conn.request("GET", "http://api.sportradar.us/nba/trial/v8/en/games/2023/REG/schedule.xml?api_key=tnsedg3csjfdvbn2ym8262np")
res = conn.getresponse()
data = res.read()
full_data = data.decode("utf-8")

fd_df_dict = {
    "DATA": [full_data],
    "YEAR": [2023]
}

fd_df = pd.DataFrame(fd_df_dict)

con = sqlite3.connect("./RAD.sqlite3")
fd_df.to_sql("RAW_DATA_NBA_SCHED_CUR", con=con, if_exists='replace', index=False)
con.close()

In [3]:
import xml.etree.ElementTree as ET
import sqlite3
import pandas as pd

year_list = ["2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]

def get_year_id(year):
    con = sqlite3.connect("./RAD.sqlite3")
    test = pd.read_sql("SELECT * FROM RAW_DATA_NBA_SCHED_CUR WHERE YEAR = '2023'", con=con)
    con.close()

    xml_data = test["DATA"].values.tolist()
    xml_cont = xml_data[0].strip()

    with open("./xml_recent_data.xml", 'w', encoding='utf-8') as new_file:
        new_file.write(xml_cont)

def remove_namespace(tree):
    for elem in tree.iter():
        if '}' in elem.tag:
            elem.tag = elem.tag.split('}', 1)[1]

def get_data_from_xml():
    xml_file = "./rec_game_output.xml"

    tree = ET.parse(xml_file)
    root = tree.getroot()

    ids = []
    for element in root.iter('game'):
        id_value = element.get('id', None)
        ids.append(id_value)
    
    home_teams = []
    for element in root.iter('home'):
        home_value = element.get('name', None)
        home_teams.append(home_value)

    away_teams = []
    for element in root.iter('away'):
        away_value = element.get('name', None)
        away_teams.append(away_value)

    home_abbrev = []
    for element in root.iter('home'):
        home_abb_value = element.get('alias', None)
        home_abbrev.append(home_abb_value)

    away_abbrev = []
    for element in root.iter('away'):
        abb_value = element.get('alias', None)
        away_abbrev.append(abb_value)

    date_list = []
    for element in root.iter('game'):
        date_value = element.get('scheduled', None)
        date_list.append(date_value)

    surface_list = []
    for element in root.iter('venue'):
        surface_value = element.get('name', None)
        surface_list.append(surface_value)

    roof_list = []
    for element in root.iter('game'):
        roof_value = element.get('status', None)
        roof_list.append(roof_value)
            
    home_scoring_list = []
    for element in root.iter('game'):
        hs_value = element.get('home_points', None)
        home_scoring_list.append(hs_value)

    away_scoring_list = []
    for element in root.iter('game'):
        as_value = element.get('away_points', None)
        away_scoring_list.append(as_value)

    data_dict = {
        "GAME_ID": ids,
        "HOME_TEAM": home_teams,
        "AWAY_TEAM": away_teams,
        "HOME_ALIAS": home_abbrev,
        "AWAY_ALIAS": away_abbrev,
        "GAME_DATE": date_list,
        "VENUE_NAME": surface_list,
        "GAME_STATUS": roof_list,
        "HOME_POINTS": home_scoring_list,
        "AWAY_POINTS": away_scoring_list,
    }

    df = pd.DataFrame(data_dict)

    return df

def inject_df(df):
    con = sqlite3.connect("./RAD.sqlite3")
    df.to_sql("NBA_GAME_DETAILS_CUR", con=con, if_exists='replace', index=False)
    con.close()

def main():
    for i in range(7, 8):
        get_year_id(year_list[i])

        tree = ET.parse('./xml_recent_data.xml')
        remove_namespace(tree)
        tree.write('./rec_game_output.xml')
        df = get_data_from_xml()
        inject_df(df)

if __name__ == "__main__":
    main()
